In [ ]:
!nvidia-smi

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls /kaggle/input/optic-disk-resized-segment/

In [ ]:
!sudo rm -rf /kaggle/input/*

In [ ]:
!zip -r9 x_ray_train.zip /kaggle/input/chest-xray-pneumonia/chest_xray/train/*

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
! git clone https://github.com/taj1290/eye_segmentation.git

In [ ]:
%cd eye_segmentation/

In [ ]:
!apt-get install unrar
!mkdir training
!unrar e  training.rar training/
!mkdir testing
!unrar e  test.rar testing/

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
import shutil
tf.__version__

'2.6.4'

In [2]:
import cv2
import numpy as np
import random

def rotateImage(image, angle):
    image_center=tuple(np.array(image.shape[1::-1])/2)
    rot_mat=cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result=cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def bruit(image):
    h, w, c=image.shape
    n=np.random.randn(h, w, c)*random.randint(5, 30)
    return np.clip(image+n, 0, 255).astype(np.uint8)

def change_gamma(image, alpha=1.0, beta=0.0):
    return np.clip(alpha*image+beta, 0, 255).astype(np.uint8)

def color(image, alpha=20):
    n=[random.randint(-alpha, alpha), random.randint(-alpha, alpha),random.randint(-alpha, alpha)]
    return np.clip(image+n, 0, 255).astype(np.uint8)

def random_change(image):
    if np.random.randint(2):
        img=change_gamma(image, random.uniform(0.8, 1.2), np.random.randint(100)-50)
    if np.random.randint(2):
        img=bruit(image)
    if np.random.randint(2):
        img=color(image)
    return image

In [ ]:
#import model
#import traitement_images as ti
# kaggle/input/optic-disk-resized-segment
dir_images='training/images/'
dir_mask  ='training/1st_manual/'


try:
  os.mkdir(dir_mask)
except:
  pass

try:
  os.mkdir(dir_images)
except:
  pass


for i in os.listdir('training'):
  # print(i)
  if os.path.isdir(os.path.join('training',i)):
    continue
  if i.endswith(".tif"):
    shutil.copy(os.path.join("training", i ), os.path.join(dir_images ,i))
  else:
    shutil.copy(os.path.join("training", i ), os.path.join(dir_mask,i ))

if not os.path.isdir(dir_images):
    print("dir_images=",dir_images)
    quit("The directory {} don't exist !".format(dir_images))
if not os.path.isdir(dir_mask):
    print("dir_images=",dir_mask)
    quit("The directory {} don't exist !".format(dir_mask))

In [3]:
dir_all_images='/kaggle/input/optic-disk-resized-segment/resized_training'
# dir_mask  ='training/1st_manual/'

In [4]:
list_file=os.listdir(dir_all_images)
len(list_file)

140

In [5]:
list_file[:20]

['IDRiD_52.jpg',
 'IDRiD_23_OD.jpg',
 'IDRiD_36.jpg',
 'IDRiD_13.jpg',
 'IDRiD_69_OD.jpg',
 'IDRiD_16.jpg',
 'IDRiD_53_OD.jpg',
 'IDRiD_44_OD.jpg',
 'IDRiD_23.jpg',
 'IDRiD_26.jpg',
 'IDRiD_08_OD.jpg',
 'IDRiD_19.jpg',
 'IDRiD_11_OD.jpg',
 'IDRiD_46_OD.jpg',
 'IDRiD_52_OD.jpg',
 'IDRiD_08.jpg',
 'IDRiD_05_OD.jpg',
 'IDRiD_20.jpg',
 'IDRiD_12.jpg',
 'IDRiD_33.jpg']

In [8]:
list_file = [image_path for image_path in os.listdir(dir_all_images) if 'OD' not in image_path]

In [14]:
len(list_file)

70

In [12]:
tab_images=[]
tab_masks=[]

In [26]:
for image_path in list_file[35:40]:
    print(f"Image path is {image_path}")
    img_orig=cv2.imread(os.path.join(dir_all_images, image_path))
    tab_images.append(img_orig)
    name, ext = os.path.splitext(image_path)
    num=image_path.split('_')[0]
    file_mask= os.path.join(dir_all_images, f"{name}_OD{ext}")
    #file_mask=fichier[-4]+"_mask.gif"
    print(f"Annotaion Image path is {file_mask}")
    if not os.path.isfile(file_mask):
        quit("Mask of {} don't exist in {}".format(file_mask, dir_mask))
#     img_mask_orig=np.array(Image.open(file_mask))
    img = cv2.imread(file_mask, 0)
    # gray = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)
    ret,img_mask_orig = cv2.threshold(img,70,255,cv2.THRESH_BINARY)
#     print(img_mask_orig.shape)
#     break
    tab_masks.append(img_mask_orig)

    for angle in range(0, 360, 30):
        img_r=rotateImage(img_orig, angle)
        img=img_r.copy()
        img=random_change(img)
        tab_images.append(img)
        img_mask=rotateImage(img_mask_orig, angle)
        tab_masks.append(img_mask)
        
        img=cv2.flip(img_r, 0)
        img=random_change(img)
        tab_images.append(img)
        img_m=cv2.flip(img_mask, 0)
        tab_masks.append(img_m)

        img=cv2.flip(img_r, 1)
        img=random_change(img)
        tab_images.append(img)
        img_m=cv2.flip(img_mask, 1)
        tab_masks.append(img_m)

        img=cv2.flip(img_r, -1)
        img=random_change(img)
        tab_images.append(img)
        img_m=cv2.flip(img_mask, -1)
        tab_masks.append(img_m)

Image path is IDRiD_06.jpg
Annotaion Image path is /kaggle/input/optic-disk-resized-segment/resized_training/IDRiD_06_OD.jpg
Image path is IDRiD_51.jpg
Annotaion Image path is /kaggle/input/optic-disk-resized-segment/resized_training/IDRiD_51_OD.jpg
Image path is IDRiD_65.jpg
Annotaion Image path is /kaggle/input/optic-disk-resized-segment/resized_training/IDRiD_65_OD.jpg
Image path is IDRiD_66.jpg
Annotaion Image path is /kaggle/input/optic-disk-resized-segment/resized_training/IDRiD_66_OD.jpg
Image path is IDRiD_35.jpg
Annotaion Image path is /kaggle/input/optic-disk-resized-segment/resized_training/IDRiD_35_OD.jpg


In [27]:
len(tab_images), len(tab_masks)

(1960, 1960)

In [22]:
cv2.imwrite("test.png", tab_masks[0])

True

In [ ]:
tab_images=np.array(tab_images, dtype=np.float32)/255
tab_masks =np.array(tab_masks,  dtype=np.float32)[:, :, :]/255

In [ ]:
tab_images.shape, tab_masks.shape

In [ ]:
train_images, test_images, train_masks, test_masks=train_test_split(tab_images, tab_masks, test_size=0.05)

del tab_images
del tab_masks

In [ ]:
train_images.shape, test_images.shape, train_masks.shape, test_masks.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
# 576, 560
def model(nbr):
    entree=layers.Input(shape=(560, 576, 3), dtype='float32')

    result=layers.Conv2D(nbr, 3, activation='relu', padding='same')(entree)
    result=layers.BatchNormalization()(result)
    result=layers.Conv2D(nbr, 3, activation='relu', padding='same')(result)
    result1=layers.BatchNormalization()(result)

    result=layers.MaxPool2D()(result1)

    result=layers.Conv2D(2*nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)
    result=layers.Conv2D(2*nbr, 3, activation='relu', padding='same')(result)
    result2=layers.BatchNormalization()(result)

    result=layers.MaxPool2D()(result2)

    result=layers.Conv2D(4*nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)
    result=layers.Conv2D(4*nbr, 3, activation='relu', padding='same')(result)
    result3=layers.BatchNormalization()(result)

    result=layers.MaxPool2D()(result3)

    result=layers.Conv2D(4*nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)
    result=layers.Conv2D(4*nbr, 3, activation='relu', padding='same')(result)
    result4=layers.BatchNormalization()(result)

    result=layers.MaxPool2D()(result4)

    result=layers.Conv2D(8*nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)
    result=layers.Conv2D(4*nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)

    result=layers.UpSampling2D()(result)
    result=tf.concat([result, result4], axis=3)

    result=layers.Conv2D(8*nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)
    result=layers.Conv2D(4*nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)

    result=layers.UpSampling2D()(result)
    result=tf.concat([result, result3], axis=3)
    
    result=layers.Conv2D(4*nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)
    result=layers.Conv2D(2*nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)

    result=layers.UpSampling2D()(result)
    result=tf.concat([result, result2], axis=3)
    
    result=layers.Conv2D(2*nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)
    result=layers.Conv2D(nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)
    
    result=layers.UpSampling2D()(result)
    result=tf.concat([result, result1], axis=3)
    
    result=layers.Conv2D(nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)
    result=layers.Conv2D(nbr, 3, activation='relu', padding='same')(result)
    result=layers.BatchNormalization()(result)

    sortie=layers.Conv2D(1, 1, activation='sigmoid', padding='same')(result)

    model=models.Model(inputs=entree, outputs=sortie)
    print("model initialized")
    return model

In [ ]:
my_model=model(64)

my_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
history = my_model.fit(train_images,
             train_masks,
             epochs=6,
             batch_size=4,
             validation_data=(test_images, test_masks))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(os.path.join("optic-disk-accuracy-chart.png"))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(os.path.join("optic-disk-loss-chart.png"))

In [ ]:
dir_test_images='/kaggle/input/optic-disk-resized-segment/resized_testing/'

tab_test_images=[]
tab_files=[]
for image_name in os.listdir(dir_test_images):
    if 'OD' in image_name:
      continue
    print(f"Test image path is {dir_test_images+image_name}")
    img=cv2.imread(dir_test_images+image_name)
    tab_test_images.append(img)
    name, ext = os.path.splitext(image_name)
    tab_files.append(name)

tab_test_images=np.array(tab_test_images, dtype=np.float32)/255
tab_files=np.array(tab_files)

In [ ]:
tab_files

In [ ]:
!mkdir predictions

In [ ]:
for id in range(len(tab_test_images)):
    prediction=my_model.predict(np.array([tab_test_images[id]]))
    mask=prediction[0]*255
    print(mask.shape)
#     break
    print(f'{"predictions/"+str(tab_files[id])+".png"}')
    cv2.imwrite("predictions/"+str(tab_files[id])+".png", mask)

In [ ]:
!mkdir weights

In [ ]:
my_model.save("weights")

In [ ]:
!zip -r9 optic_disk_segment_weights.zip weights/*

In [ ]:
!pwd

In [ ]:
!ls ../predictions/

In [ ]:
%cd ../
!ls

In [ ]:
!zip -r9 predictions.zip predictions/*

In [ ]:
!ls eye_segmentation/

In [ ]:
img_path = "/kaggle/input/optic-disk-resized-segment/resized_training/IDRiD_52_OD.jpg"

In [ ]:
img_mask_orig=np.array(Image.open(img_path).convert('L'))
print(img_mask_orig.shape)

In [ ]:
val = []
for i in range(560):
    for j in range(576):
        val.append(img_mask_orig[i][j])

In [ ]:
img = cv2.imread(img_path, 0)
# gray = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(img,70,255,cv2.THRESH_BINARY)
thresh.shape

In [ ]:
val = []
for i in range(560):
    for j in range(576):
        val.append(thresh[i][j])

In [ ]:
set(val)